In [47]:
%pip install psycopg2-binary pymongo

Note: you may need to restart the kernel to use updated packages.


In [44]:
import psycopg2
from pymongo import MongoClient
from bson import ObjectId
from datetime import datetime


conn = psycopg2.connect(
    dbname="institute",
    user="postgres",
    password="jaishreeram",
    host="localhost",
    port="5432"
)
cur = conn.cursor()

mongo_client = MongoClient('mongodb://localhost:27017/')
mongo_db = mongo_client['mongo_institute']

def extract_all_data():
    cur.execute("SELECT * FROM Departments")
    departments = cur.fetchall()

    cur.execute("SELECT * FROM Students")
    students = cur.fetchall()

    cur.execute("SELECT * FROM Instructors")
    instructors = cur.fetchall()

    cur.execute("SELECT * FROM Courses")
    courses = cur.fetchall()

    cur.execute("SELECT * FROM Enrollments")
    enrollments = cur.fetchall()

    return {
        "departments": departments,
        "students": students,
        "instructors": instructors,
        "courses": courses,
        "enrollments": enrollments
    }


In [45]:

def transform_data(postgres_data):
    department_mongo = {}
    instructor_mongo = {}
    course_mongo = {}
    student_mongo = {}
    departments = []
    for dept in postgres_data['departments']:
        department_doc = {
            "_id": dept[0],  
            "department_name": dept[1],  
            "instructors": [], 
            "students": []    
        }
        department_mongo[dept[0]] = department_doc['_id']
        departments.append(department_doc)

    instructors = []
    for inst in postgres_data['instructors']:
        instructor_doc = {
            "_id": inst[0],
            "name": inst[1],
            "department": department_mongo[inst[2]],
            "courses": [] 
        }
        instructor_mongo[inst[0]] = instructor_doc["_id"]
        instructors.append(instructor_doc)

    students = []
    for student in postgres_data['students']:
        student_doc = {
            "_id": student[0],   
            "name": student[1],
            "department": department_mongo[student[2]],  
            "enrollments": []  
        }
        student_mongo[student[0]] = student_doc['_id']
        for dept in departments:
            if dept["_id"] == department_mongo[student[2]]:
                dept["students"].append({
                    "_id": student_doc["_id"],
                    "name": student[1]
                })
        students.append(student_doc)

    courses = []
    for course in postgres_data['courses']:
        course_doc = {
            "_id": course[0],
            "course_name": course[1],
            "department": department_mongo[course[2]],
            "instructor": instructor_mongo[course[3]],
            "is_core": course[4],
            "enrollments": []
        }
        course_mongo[course[0]] = course_doc["_id"]

        for instructor in instructors:
            if instructor["_id"] == course_doc["instructor"]:
                instructor["courses"].append({
                    "_id": course_doc["_id"],
                    "course_name": course_doc["course_name"]
                })

        courses.append(course_doc)

    for enrollment in postgres_data['enrollments']:
        student_id = enrollment[1]
        course_id = enrollment[2]
        course_doc = next((c for c in courses if c["_id"] == course_mongo[course_id]), None)
        student_doc = next((s for s in students if s["_id"] == student_mongo[student_id]), None)
        if course_doc and student_doc:
            enrollment_date = datetime.combine(enrollment[3], datetime.min.time())

            enrollment_doc = {
                "_id": student_doc["_id"],
                "name": student_doc["name"],
                "enrollment_date": enrollment_date,  
                "semester": enrollment[4]
            }
            course_doc["enrollments"].append(enrollment_doc)
            student_doc["enrollments"].append({
                "_id": course_doc["_id"],
                "enrollment_date": enrollment_date, 
                "semester": enrollment[4]
            })

    return {
        "departments": departments,
        "students": students,
        "instructors": instructors,
        "courses": courses
    }

def load_data_to_mongodb(transformed_data):
    client = MongoClient("mongodb://localhost:27017/")
    db = client.mongo_institute

    db.departments.insert_many(transformed_data['departments'])
    db.students.insert_many(transformed_data['students'])
    db.instructors.insert_many(transformed_data['instructors'])
    db.courses.insert_many(transformed_data['courses'])

    print("Data successfully migrated to MongoDB!")


In [46]:
def migrate_rdbms_to_mongodb():
    postgres_data = extract_all_data()

    transformed_data = transform_data(postgres_data)

    load_data_to_mongodb(transformed_data)
    
    cur.close()
    conn.close()
    mongo_client.close()

migrate_rdbms_to_mongodb()

Data successfully migrated to MongoDB!
